In [1]:
from pyspark.sql import SparkSession
import os
from search import Search
from plotting import Plotting

os.environ["PYTHONUNBUFFERED"] = "1"
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"

In [2]:
def read(spark):
    datasets = {}
    for dataset in os.listdir("./ml-latest-small"):
        data = spark.read.option("header", True).format("csv").load(os.path.join(os.getcwd(),"ml-latest-small",dataset))
        datasets[dataset.split('.')[0]]=data
    return datasets

In [3]:
spark = SparkSession.builder.master("local").appName("Movies").config("conf-key", "conf-value").getOrCreate()
datasets = read(spark)

In [4]:
plotting = Plotting(datasets, spark)
search = Search(datasets, spark)

Search user by id, show the number of movies/genre that he/she has watched

In [5]:
# search.search_user_movies('1')

In [6]:
# search.search_user_genre('1').show()

Given a list of users, search all movies watched by each user

In [7]:
# [i.show() for i in search.search_users_movies(["1","2","3"])]

Search movie by id/title, show the average rating, the number of users that have
watched the movie

In [8]:
# search.search_movie(id="1").show()

In [9]:
# search.search_movie(name="Toy Story").show()

Search genre, show all movies in that genre

In [10]:
# search.search_genre("Action").show(10,False)

Given a list of genres, search all movies belonging to each genre

In [11]:
# [i.show(3, False) for i in search.search_genres(["Action","Romance"])]
# search.search_genres(["Action","Romance"]).show()

Search movies by year

In [12]:
# search.search_movie_year("1995").show(10,False)

List the top n movies with highest rating, ordered by the rating

In [13]:
# search.list_rating(5)

List the top n movies with the highest number of watches, ordered by the number of
watches

In [14]:
# search.list_watches(5)

Find the favourite genre of a given user, or group of users. Consider and justify how
you will define ‘favourite’.

In [15]:
# search.search_user_favourites("1").show(10,False)

Compare the movie tastes of two users. Consider and justify how you will compare
and present the data.

In [16]:
search.compare_users("1", "2")

search.compare_users("477", "610")

search.compare_users("1", "1")

user a:  232
+-------+-------+
|movieId|ratingA|
+-------+-------+
|      1|    4.0|
|      3|    4.0|
|      6|    4.0|
|     47|    5.0|
|     50|    5.0|
+-------+-------+

user b:  29
+-------+-------+
|movieId|ratingB|
+-------+-------+
|    318|    3.0|
|    333|    4.0|
|   1704|    4.5|
|   3578|    4.0|
|   6874|    4.0|
+-------+-------+

+-------+-------+-------+
|movieId|ratingA|ratingB|
+-------+-------+-------+
|    333|    5.0|    4.0|
|   3578|    5.0|    4.0|
+-------+-------+-------+

4.366379310344827
3.9482758620689653
+-------+-------+-------+--------------------+--------------------+
|movieId|ratingA|ratingB|ratingA-avg(ratingA)|ratingB-avg(ratingB)|
+-------+-------+-------+--------------------+--------------------+
|    333|    5.0|    4.0|  0.6336206896551726|  0.0517241379310347|
|   3578|    5.0|    4.0|  0.6336206896551726|  0.0517241379310347|
+-------+-------+-------+--------------------+--------------------+

+-------------------+
|          numerator|
+-

DataFrame[similarity: double]

In [17]:
# plotting.gen_movies_report()

In [18]:
# plotting.gen_user_report("1")

Plot the most watched movies in a given year

In [19]:
# plotting.generic_plot(search.filter_year("1995", 10,"watches"), 'watches')

Plot the most watched movies in a given decade

In [20]:
# plotting.generic_plot(search.filter_decade("199", 10,"watches"), 'watches')

Plot the most watched movies of each decade

In [21]:
# plotting.generic_plot(search.top_each_decade('watches'), 'watches')

Plot the most watched movies of all time

In [22]:
# plotting.plot_generic_plotwatches(search.list_watches(10), 'watches')

Plot the highest rated movies in a given year

In [23]:
# plotting.generic_plot(search.filter_year("1995", 10,"avg(rating)"),avg(rating))

Plot the highest rated movies in a given decade

In [24]:
# plotting.generic_plot(search.filter_decade("199", 10,"avg(rating)"),avg(rating))

Plot the highest rated movies of each decade

In [25]:
# plotting.generic_plot(search.top_each_decade('avg(rating)'),avg(rating))

Plot the highest rated movies of all time

In [26]:
# plotting.generic_plot(search.list_rating(10))

In [27]:
# plotting.gen_movie_wordcloud("Pulp Fiction")

Recommend movies for a set of users

In [28]:
# search.recommend_n_movies_for_users(5, ["1", "3", "47", "235"]).show(truncate=False)

In [29]:
# search.recommend_n_movies_for_users(5, ["1"]).show(truncate=False)
